# Advanced Free Rag

In [6]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

## Define the functions

In [9]:
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline,
    EmbeddingsFilter,
    LLMChainFilter,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import (
    EmbeddingsRedundantFilter,
    LongContextReorder,
)
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.text_splitter import SemanticChunker
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain import hub
from typing import List

# Remember you have to use the same embedding your database was built with

llm_model = "gpt-3.5-turbo"
embed_model = (
    "text-embedding-3-large"  # This embed model returns a vector of 3056 dimensions
)

def get_llm(model: str = llm_model):
    return ChatOpenAI(name=model, temperature=0)


def get_embedding(model: str = embed_model):
    embedding_function = OpenAIEmbeddings(model=model, max_retries=3)
    return embedding_function


def semantic_chunk_documents(documents: list[Document], threshold: int = 60):
    text_splitter = SemanticChunker(
        embeddings=get_embedding(),
        breakpoint_threshold_type="percentile",  # "percentile" "standard_deviation", "interquartile",
        breakpoint_threshold_amount=threshold,
    )
    documents = text_splitter.split_documents(documents)
    return documents


def delete_collection(collection="Rdi_vectorDb"):
    db = Chroma(
        collection_name=collection,
        embedding_function=get_embedding(model=embed_model),
        persist_directory="./temp/Databases",
    )
    return db.delete_collection()


def add_to_chroma(
    chunks: List[str], metadatas: List[dict], ids: List[str], collection="Rdi_vectorDb"
) -> List[str]:
    db = Chroma(
        collection_name=collection,
        embedding_function=get_embedding(model=embed_model),
        persist_directory="./temp/Databases",
    )
    return db.add_texts(chunks, metadatas, ids)


def get_db(collection="Rdi_vectorDb"):
    return Chroma(
        persist_directory="./temp/Databases",
        collection_name=collection,
        embedding_function=get_embedding(model=embed_model),
    )


def get_retriever(vectorstore, local_llm):
    import logging
    logging.basicConfig()
    logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
    simple_retriever = vectorstore.as_retriever(
        search_function="similarity",  # Can be "similarity" (default), "mmr", or "similarity_score_threshold"
        search_kwargs={"k": 5},
    )  # k=5 returns the 5 most relevant chunks in the vector database
    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=simple_retriever, llm=local_llm
    )

    # Split the chunks even more
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=300, chunk_overlap=0, separators=[". "]
    )

    # Remove the redundant documents by using an EmbeddingsRedundantFilter
    redundant_filter = EmbeddingsRedundantFilter(
        embeddings=get_embedding(model=embed_model)
    )

    # The EmbeddingsFilter # provides a cheaper and faster option by embedding the documents and
    # return only the ones with sufficiently similar embeddings to the query.
    relevant_filter = EmbeddingsFilter(
        embeddings=get_embedding(model=embed_model), similarity_threshold=0.2
    )

    reorder_filter = LongContextReorder()

    llm_filter = LLMChainFilter.from_llm(
        local_llm
    )  # THIS ONE USES LLM to check similarity, it will slow the process

    pipeline_compressor = DocumentCompressorPipeline(
        transformers=[
            splitter,
            redundant_filter,
            relevant_filter,
            reorder_filter,
            llm_filter,
        ]
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=pipeline_compressor, base_retriever=multiquery_retriever
    )

    return compression_retriever


def rag(question: str, collection="Rdi_vectorDb", stream: bool = False):
    local_llm = get_llm(model=llm_model)
    vectorstore = get_db(collection=collection)
    prompt = hub.pull("rlm/rag-prompt") #https://smith.langchain.com/hub/rlm/rag-prompt

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    chain = (
        RunnableParallel(
            {
                "context": get_retriever(vectorstore, local_llm) | format_docs,
                "question": RunnablePassthrough(),
            }
        )
        | prompt
        | local_llm
        | StrOutputParser()
    )

    if not stream:
        print(chain.invoke(question))
    else:
        for result in chain.stream(question):
            print(result, end="", flush=True)

ModuleNotFoundError: No module named 'pydantic.v1'

In [ ]:
# What is Hub pull ?
# #https://smith.langchain.com/hub
prompt = hub.pull("rlm/rag-prompt")
from rich import print

print(prompt)

ChatPromptTemplate(
    input_variables=['context', 'question'],
    metadata={
        'lc_hub_owner': 'rlm',
        'lc_hub_repo': 'rag-prompt',
        'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                template="You are an assistant for question-answering tasks. Use the following pieces of retrieved 
context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences 
maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
            )
        )
    ]
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing import cast
prompt = cast(ChatPromptTemplate, prompt)
print(prompt.partial(question="What is RDI?"))

ChatPromptTemplate(
    input_variables=['context'],
    partial_variables={'question': 'What is RDI?'},
    metadata={
        'lc_hub_owner': 'rlm',
        'lc_hub_repo': 'rag-prompt',
        'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                template="You are an assistant for question-answering tasks. Use the following pieces of retrieved 
context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences 
maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
            )
        )
    ]
)

## Load the Data

In [ ]:
#Load your data
delete_collection(collection="Rdi_vectorDb")

#HTML
from langchain_community.document_loaders import WebBaseLoader
from rich import print

loader_HTML = WebBaseLoader(web_paths=["https://www.rdisoftware.com/"])
docsWeb = loader_HTML.load() #load
print(docsWeb)
#Expected execution time: 1s

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[
    Document(
        page_content="\n\n\n\n\nRDI 
Software\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    \n  
\n\n\n\n\nLinkedin\nInstagram\nGlassdoor\n\n\n\n  Home  \n Discover RDI \n Our Services \n Careers \n Testimonials 
\n Contact Us \n\n\n\n\n\n\n\n\n\n \nmore than just a technology company\na PEOPLE company\n DISCOVER 
\n\n\n\n\n\n\n\n\nWe proudly serve the leading global foodservice retailer with a focus on improving the overall 
crew and customer experience, resulting in improved operations in the restaurants.\n\n\n\n\n\n\n\n\nDiscover 
RDI\n\n\n\n  \n\nRDI has 20+ years of history working as a strategic partner of a major global QSR (Quick Service 
Restaurant) retailer.\n\n\n\n\n\n  \n\nRDI delivers POS services (point-of-sale) and related software applications 
that focus on the overall crew and customer experience, resulting in improved operations to the fastest growing 
food restaurant chain over 120 countries and over 37,500 locations worldwide.RDI joined Capgemini group in 2017, 
which has 270,000 collaborators in almost 50 countries.\n\n\n\n\n\n  \n\nOur Development Centers are in Sao Paulo 
(Brazil), Budapest and Debrecen (Hungary).\n\n\n\n\n\n\n\n\n\n\n\n  120\ncountries deployed\n\n  
37,500\nrestaurants running our products\n\n  1.7\nmillion users\n\n  2\ndevelopment 
centers\n\n\n\n\n\n\n\n\n\nPurpose and Values\n\n\n\nOur Purpose\nSupport our partner by increasing product 
deliveries and leveraging high-end technology to transform customer experience.\n\n\n\n\nOur Vision\nProvide 
remarkable experience for our PEOPLE and our partner!\n\n\n\n\n\n\n\n\n\n\nOur Powerful 
Values\n\n\n\n\n\n\n\n\n\n\n\n\nDiversity and Inclusion\n\n\nRDI values the plurality of race, color, religion, 
gender, gender identity, nationality, sexual orientation, disabilities and age.\n              Be plural is value 
and respect the differences. At RDI you are free to be you!\n\n\n\n\n\n\n\n\n\nOur Services\n\n\n\n  \n\nSoftware 
Development\nWe perform all phases of the development life cycle across crew and customer-facing restaurant 
applications and work closely with our client to bring innovative solutions to life in the 
restaurants.\n\xa0\n\n\n\n  \n\nTesting and Support\nWe perform unit and full regression tests of our software to 
ensure that existing and new functionality is working as defined. We review, investigate and resolve issues 
reported through the test environments and live restaurants to ensure that the application supports the restaurant 
business 24x7.\n\n\n\n\n\n  \n\nRelease Management\nRDI provides software release management services for a tightly
integrated suite of applications. Our  service maintains the development and delivery schedules for products, 
identifying any potential conflicts or opportunities to speed delivery.\n\n\n\n\n\n  \n\nLocalization services\nThe
company provides documentation and localization services for new applications developed and delivered to the 
markets.\n\n\n\n\n\n\n\n\n\n\nCareers\n\n\n If you love the idea of leveraging your technical and business skills 
to help drive growth in the Quick Service Restaurantindustry through the innovative use of new technology, join 
us.\n            RDI is an equal opportunity employer and is committed to a diverse and inclusive workforce. 
\n\n\n\n\n\n\n\n\nJoin Us\n\nWe have open positions in the offices of Brazil and Hungary:\nApply 
Now\n\n\n\n\n\n\nBenefits & How we hire \nBenefits - RDI’s collaborative and flexible work environment and 
high-performance team delivers innovative solutions to our clients. Our benefit package includes a range of 
market-relevant benefits to help you manage the demands of your career and home life.\nHow we hire - Our process is
simple; the path to getting hired usually involves a first conversation with a recruiter, a technical evaluation 
and an interview at one of our global offices with the hiring Manager and HR. \n\n\n\n\n\n\n \nJoin 
Us\nRevolutionizing restauran

## Chunk the Data into smaller semantic similar pieces

In [ ]:
#Add to the vector database
chunks = semantic_chunk_documents(docsWeb)
print("total chunks: ", len(chunks))
print(chunks)

total chunks:  21

[
    Document(
        page_content='\n\n\n\n\nRDI 
Software\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    \n  
\n\n\n\n\nLinkedin\nInstagram\nGlassdoor\n\n\n\n  Home  \n Discover RDI \n Our Services \n Careers \n Testimonials 
\n Contact Us \n\n\n\n\n\n\n\n\n\n \nmore than just a technology company\na PEOPLE company\n DISCOVER 
\n\n\n\n\n\n\n\n\nWe proudly serve the leading global foodservice retailer with a focus on improving the overall 
crew and customer experience, resulting in improved operations in the restaurants. Discover RDI\n\n\n\n  \n\nRDI 
has 20+ years of history working as a strategic partner of a major global QSR (Quick Service Restaurant) retailer. 
RDI delivers POS services (point-of-sale) and related software applications that focus on the overall crew and 
customer experience, resulting in improved operations to the fastest growing food restaurant chain over 120 
countries and over 37,500 locations worldwide.RDI joined Capgemini group in 2017, which has 270,000 collaborators 
in almost 50 countries. Our Development Centers are in Sao Paulo (Brazil), Budapest and Debrecen (Hungary).',
        metadata={
            'source': 'https://www.rdisoftware.com/',
            'title': 'RDI Software',
            'description': 'RDI Software - Part of Capgemini',
            'language': 'No language found.'
        }
    ),
    Document(
        page_content='120\ncountries deployed\n\n  37,500\nrestaurants running our products\n\n  1.7\nmillion 
users\n\n  2\ndevelopment centers\n\n\n\n\n\n\n\n\n\nPurpose and Values\n\n\n\nOur Purpose\nSupport our partner by 
increasing product deliveries and leveraging high-end technology to transform customer experience. Our 
Vision\nProvide remarkable experience for our PEOPLE and our partner!',
        metadata={
            'source': 'https://www.rdisoftware.com/',
            'title': 'RDI Software',
            'description': 'RDI Software - Part of Capgemini',
            'language': 'No language found.'
        }
    ),
    Document(
        page_content='Our Powerful Values\n\n\n\n\n\n\n\n\n\n\n\n\nDiversity and Inclusion\n\n\nRDI values the 
plurality of race, color, religion, gender, gender identity, nationality, sexual orientation, disabilities and age.
Be plural is value and respect the differences.',
        metadata={
            'source': 'https://www.rdisoftware.com/',
            'title': 'RDI Software',
            'description': 'RDI Software - Part of Capgemini',
            'language': 'No language found.'
        }
    ),
    Document(
        page_content='At RDI you are free to be you! Our Services\n\n\n\n  \n\nSoftware Development\nWe perform all
phases of the development life cycle across crew and customer-facing restaurant applications and work closely with 
our client to bring innovative solutions to life in the restaurants.',
        metadata={
            'source': 'https://www.rdisoftware.com/',
            'title': 'RDI Software',
            'description': 'RDI Software - Part of Capgemini',
            'language': 'No language found.'
        }
    ),
    Document(
        page_content='Testing and Support\nWe perform unit and full regression tests of our software to ensure that
existing and new functionality is working as defined.',
        metadata={
            'source': 'https://www.rdisoftware.com/',
            'title': 'RDI Software',
            'description': 'RDI Software - Part of Capgemini',
            'language': 'No language found.'
        }
    ),
    Document(
        page_content='We review, investigate and resolve issues reported through the test environments and live 
restaurants to ensure that the application supports the restaurant business 24x7. Release Management\nRDI provides 
software release management services for a tightly integrated suite of applications. Our  service maintains the 
development and delivery schedules for products, identifying any potential conflicts or opportunities to s

## Add the Chunks to the Vector Database

In [ ]:
import hashlib
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks]
ids = [hashlib.sha256(text.encode()).hexdigest() for text in texts]

delete_collection(collection="Rdi_vectorDb")
added_ids = add_to_chroma(texts, metadatas, ids, collection="Rdi_vectorDb")
print("total added ids: ", len(added_ids))
print(added_ids)

total added ids:  21

[
    'ce9f4302b74be8bc2ecdaee50e1cb222d68335e404b499901e37ffdaae187cf4',
    '69771a751716af65e7d5bfc97347326c37bac8179d7fab182aec3fd2a1a26709',
    '661791684571a415f1d0c3c3c092b396c80b1659b45d7218438c67e0df6df2c5',
    '3847dab2dee57e2b8760dd8c97fa04b7fd05ff7754f8039ad6b04de52da13dae',
    '8cbdd83e0a3bbe7db3bc1f61717995566e18fbde89e80308d1842daca1c5c772',
    '6306e87788b0b3bba75df290fbb640b51e169322bc26a84916b559fa0f50e3fe',
    'd3ccdb9add7a91f723b7ae1c563518f99db8213c76ce3e58778d79da943779d6',
    '8503846a4075611737f4de8f7f54cae7617ef2d78e602407c997cab5f7d4fff3',
    '7b8eee6fced3995af83738ec093fc8a40dac9d07d41b848d86a4f46ab70f0645',
    'bdedde35b2b6f1430bc39dc3d21d74928344d1d2716b69cfb8ea6438be474a5a',
    '3fc1f93a5cd425673b0401990710e21412fa3e68ef9000fb17b02651eb284233',
    '7c0227385e9f7eeaefb5b734792c6cdf195ca1ee4fd1cd3fae06c863a511c768',
    '738a6ff205acf9a2ef5493616b0017ae655beae1839edf745bbb8f035391e4cb',
    '85e25e5e3c89ec377e5a52b29cbdc36c9cd82bc93ea703fbe7b1aa9fe498234c',
    'c3f17ecc95db4c17f866cbb638ffaddb5193c315d85d3993f00f156fc7446ef6',
    '522990ac6003212d34df2811e56cc52177c35d0e9a44e493c0a98940012f793c',
    '7a2e6e388cdafed0917df1b0731269b01854abc01753e3723c0f9eef8237f5e8',
    '6a380d550c5c48102d1755740c887837a9cd03bd5544ea2cdd58b8fce6ab1fd3',
    'e7dd05a356481a01ffed230e605db2a338d8c74180ae92065116b8d629914f5e',
    '42b1887226cf7e07a82a789dce278031c083a33ba4bc00babff3f9792bca5a39',
    'e9ef131b1eabd2a14386dc3bb15cbebab16e2ae4fff7410c7e3e4f0795c32899'
]

In [ ]:
# Do not join multiple questions in one example!!! 
# How to define the semantic similarity between questions?
rag(question="""
             1. What is RDI Software specialized in?
             2. When was RDI bought by Capgemini?
             3. Are there new openings ?
             4. What are the benefits of working in RDI?
             5. Who is Sarolta Mezei and what she said about RDI ?
             """)

#Expected execution time: 8min on local Ollama
#Expected execution time with OpenAI: 21.7s

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an outp

1. RDI Software is specialized in delivering POS services and related software applications that focus on improving
the overall crew and customer experience in restaurants.
2. RDI was bought by Capgemini in 2017.
3. Yes, there are new openings at RDI in the offices of Brazil and Hungary.

None

In [ ]:
questions = [
     "What is RDI Software specialized in ?",
     "When was RDI bought by Capgemini?",
     "Are there new openings ?",
     "What are the benefits of working in RDI?",
     "Who is Sarolta Mezei and what she said about RDI ?"
]

answers = []
for i,question in enumerate(questions):
    rag(question)
    
#Expected execution time: 1min 20s

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

RDI Software is specialized in software development for restaurant applications, particularly focusing on POS 
services and related software applications to enhance the crew and customer experience in restaurants. They also 
provide software release management services for a suite of applications to maintain development and delivery 
schedules. RDI has a history of working as a strategic partner for a major global Quick Service Restaurant 
retailer.

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

RDI was bought by Capgemini in 2017.

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

Yes, there are new openings in the offices of Brazil and Hungary. Apply now by contacting RDI HR directly at 
rdihumanresources.hr@capgemini.com. The company is looking for strong performers who love big challenges and will 
fit in with the culture at RDI.

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

The benefits of working at RDI include a collaborative and flexible work environment, a high-performance team, and 
a range of market-relevant benefits to help manage career and home life demands. Employees at RDI are valued, 
treated well, and have the opportunity to work independently with trust from employers. The company is committed to
a diverse and inclusive workforce, supports personal and professional development, and encourages employees to take
on new challenges in their careers.

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

Sarolta Mezei is a colleague at RDI Hungary who had a fantastic experience with the company. She mentioned that RDI
Hungary and her colleagues welcomed her warmly despite cultural and linguistic differences. She appreciated the 
support, opportunities for personal and professional development, and the open communication environment at RDI.

In [ ]:
rag("Who works as a Software Quality Assurance Analyst at RDI?", stream=True)
#Expected execution time: 15s

/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/darthlinuxer/AI/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly 

A

ri

ane

Med

...

Sao

Paulo

,

Brazil

.